In [1]:
import numpy as np
import pandas as pd 
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from glob import glob
import matplotlib.pyplot as plt

import utils
# from utils import load_data, get_train_val_split, get_stratified_splitter
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from itertools import combinations
from collections import defaultdict
from tqdm import tqdm
from sklearn.ensemble import StackingRegressor
import re
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error, silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

import optuna
import pickle

import shap

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\johns\\Desktop\\probstats2\\EnergyPrediction-ASHRAE\\code\\utils.py'>

In [3]:
data_dict = utils.load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


In [4]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [5]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

In [6]:
features = ['year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'air_temperature_mean_lag7', 'air_temperature_max_lag7',
       'air_temperature_min_lag7', 'air_temperature_std_lag7',
       'cloud_coverage_mean_lag7', 'cloud_coverage_max_lag7',
       'cloud_coverage_min_lag7', 'cloud_coverage_std_lag7',
       'dew_temperature_mean_lag7', 'dew_temperature_max_lag7',
       'dew_temperature_min_lag7', 'dew_temperature_std_lag7',
       'precip_depth_1_hr_mean_lag7', 'precip_depth_1_hr_max_lag7',
       'precip_depth_1_hr_min_lag7', 'precip_depth_1_hr_std_lag7',
       'sea_level_pressure_mean_lag7', 'sea_level_pressure_max_lag7',
       'sea_level_pressure_min_lag7', 'sea_level_pressure_std_lag7',
       'wind_direction_mean_lag7', 'wind_direction_max_lag7',
       'wind_direction_min_lag7', 'wind_direction_std_lag7',
       'wind_speed_mean_lag7', 'wind_speed_max_lag7', 'wind_speed_min_lag7',
       'wind_speed_std_lag7', 'log_square_feet', 'weekday', 'hour', 'day',
       'weekend', 'month', 'primary_use_enc']

In [7]:
X_full = data_dict['X_train'].loc[:, features+['meter', 'site_id', 'building_id', 'timestamp']]

In [8]:
building_cluster_mapping = pd.DataFrame()

silhouette_scores = {}
for site in X_full['site_id'].unique():
    for meter in X_full['meter'].unique():
        X = X_full[(X_full["site_id"]==site)&(X_full["meter"]==meter)]
        y = data_dict['y_train'][(X_full["site_id"]==site)&(X_full["meter"]==meter)]
        X["meter_reading"] = y

        X["YMD"] = X["timestamp"].dt.strftime("%Y-%m-%d")
        day_level_df = X.groupby(["building_id", "YMD"]).agg(meter_reading = ("meter_reading", "mean")).reset_index()
        day_level_pivot = pd.pivot_table(day_level_df, values="meter_reading", columns="YMD", index="building_id").fillna(0)
        if X.shape[0]==0:
            continue
        
        
        key = "_".join([str(site), str(meter)])
        
        silhouette_scores[key] = []
        n_buildings = day_level_pivot.shape[0]
        n_pc_range = range(2, min(n_buildings - 1, 10))
        k_range = range(2, min(5, n_buildings - 1))
        #Default best_k, best_pc values
        best_k, best_pc, best_score, best_labels = 1, 2, 0, np.ones(n_buildings)
        for n_pc in n_pc_range:
            pca = PCA(n_pc)
            t = pca.fit_transform(day_level_pivot)
            for k in k_range:
                kmeans_labels = KMeans(n_clusters=k, random_state=0).fit(t[:, :n_pc]).labels_
                ss = silhouette_score(t[:, :n_pc], kmeans_labels)
                if ss > best_score:
                    best_pc, best_k, best_score = n_pc, k, ss
                    best_labels = kmeans_labels
        
        print(site, meter, best_pc, best_k)
        
        day_level_pivot["cluster"] = best_labels
        day_level_pivot = day_level_pivot.reset_index()
        bc_mapping = day_level_pivot[["building_id", "cluster"]]
        bc_mapping["meter"] = meter
        building_cluster_mapping = pd.concat([building_cluster_mapping, bc_mapping])

1 0 2 2
1 3 2 2
2 0 2 2
2 3 2 2
2 1 2 2
3 0 2 4
4 0 2 2
5 0 2 2
6 0 2 4
6 1 2 3
6 2 2 4
7 0 2 2
7 3 2 1
7 1 2 2
7 2 2 3
8 0 2 2
9 0 2 2
9 1 2 3
9 2 2 4
10 0 2 2
10 3 2 2
10 1 2 2
11 0 2 3
11 3 2 2
11 1 2 2
12 0 2 3
13 0 2 2
13 1 2 2
13 2 2 2
14 0 2 4
14 3 2 2
14 1 2 2
14 2 2 2
15 0 2 2
15 3 2 1
15 1 2 2
15 2 2 4
0 0 2 3
0 1 2 3


In [9]:
X_full.reset_index(inplace=True)

In [10]:
import gc
gc.collect()

1299

In [11]:
del data_dict['weather_test']
del data_dict['X_train']

In [12]:
X_full = X_full.merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')

In [13]:
splitter_gen = utils.get_stratified_splitter(X_full, data_dict['y_train'])
train_index, test_index = next(splitter_gen)

In [14]:
X_test, y_test = X_full.loc[test_index, :], data_dict['y_train'][test_index]

In [15]:
model_files = os.listdir('models/v4/')# sorted(glob('models/v4/*.pkl'))
estimators = []
for model_file in model_files:
    if '.pkl' in model_file:
        estimators.append((model_file.split(".")[0], pickle.load(open('models/v4/' + model_file,'rb'))))

In [16]:
estimators_dict = dict(estimators)

In [17]:
def SHAP_individual_estimators(X_test, y_test, estimators_dict):
#     results_df = pd.DataFrame(columns=['y_true', 'y_pred'])
    for site_meter_cluster in estimators_dict.keys():

        site_id, meter, cluster = list(map(lambda x:int(x) ,site_meter_cluster.split("_")))
        # Filter input data for the specific site_id and meter
        site_meter_filter = (X_test['site_id'] == site_id) & (X_test['meter'] == meter) & (X_test['cluster'] == cluster)

        X_test_subset = X_test[site_meter_filter].drop(['meter', 'site_id', 'building_id', 'timestamp', 'cluster'], axis=1)
        
        try:
            shap_values = shap.TreeExplainer(estimators_dict[site_meter_cluster]).shap_values(X_test_subset)
            plt.title(f"Site: {site}, Meter: {meter}, Cluster: {cluster}")
            shap.summary_plot(shap_values, X_test_subset, show=False)
            plt.savefig(f"./shap_plots/{site_meter_cluster}.png", dpi=150, bbox_inches='tight')
            plt.clf()
        except:
            continue

In [18]:
SHAP_individual_estimators(X_test, y_test, estimators_dict)

<Figure size 800x950 with 0 Axes>

In [19]:
site_id, meter, cluster = list(map(lambda x:int(x), list(estimators_dict.keys())[0].split("_")))